<a href="https://colab.research.google.com/github/sumanmichael/tabulardata-classification-deeplearning/blob/main/tabulardata_classification_stacking_and_tabnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install vecstack lightgbm

In [ ]:
# import numpy as np
# import warnings
# warnings.filterwarnings("ignore")
# import pandas as pd
# df=pd.read_csv("https://dphi.s3.ap-south-1.amazonaws.com/dataset/train_age_dataset.csv")
# import matplotlib.pyplot as plt
# from sklearn.preprocessing import QuantileTransformer
# import seaborn as sns
# df.drop(['Unnamed: 0', 'userId','age_group'],axis=1,inplace=True)


# for col in list(set(df.columns)-set(['tier','gender','age_group'])):
    
#     fig, (ax1, ax2, ax3) = plt.subplots(1,3,figsize=(20,5))
#     sns.histplot(df[col],ax=ax1,kde=True)
    
#     ax1.set_ylabel("") 
#     percentiles = df[col].quantile([0.01,0.99]).values
#     df[col] = np.clip(df[col], percentiles[0], percentiles[1])
   
#     sns.histplot(df[col],ax=ax2,kde=True)
    
#     ax2.set_ylabel("")  
    
#     quantile_transformer =QuantileTransformer(random_state=0)
#     df[col] = quantile_transformer.fit_transform(np.array(df[col]).reshape(-1,1))
    
#     sns.histplot(df[col],ax=ax3,kde=True)
#     ax3.set_ylabel("")  
#     plt.show()
#     break

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv("https://dphi.s3.ap-south-1.amazonaws.com/dataset/train_age_dataset.csv")
X_final = pd.read_csv("https://dphi.s3.ap-south-1.amazonaws.com/dataset/test_age_dataset.csv")
X = df.iloc[:,:-1]
y = df.iloc[:,-1]

remCol=[]
for c in X.columns:
    if(len(X[c].unique())/len(df))>=0.99:
        remCol.append(c)
X.drop(remCol,axis=1,inplace=True)
X_final.drop(remCol,axis=1,inplace=True)

X = pd.get_dummies(X,columns=["tier","gender"],drop_first=True)
X_final = pd.get_dummies(X_final,columns=["tier","gender"],drop_first=True)

# Correlation Analysis

In [ ]:
# f, ax = plt.subplots(figsize=(11, 9))
# sns.heatmap(X.corr())

In [ ]:
import numpy as np

corr = X.corr()
columns = np.full((corr.shape[0],), True, dtype=bool)
for i in range(corr.shape[0]):
    for j in range(i+1, corr.shape[0]):
        if corr.iloc[i,j] >= 0.9:
            if columns[j]:
                columns[j] = False
selected_columns = X.columns[columns]
X=X[selected_columns]
X_final = X_final[selected_columns]

In [ ]:
# sns.heatmap(X.corr())


# Outliers

In [ ]:
# from sklearn.preprocessing import QuantileTransformer
# SUBSET = 5000
# X_Sampled = X[:SUBSET_SIZE]
# for col in X_Sampled.columns[:]:
#     plt.figure()
#     y = X_Sampled[[col]].values
#     # y = QuantileTransformer(output_distribution='normal').fit_transform(y)
#     sns.distplot(y)

In [ ]:
# pd.get_dummies(df[["tier"]],columns=["tier"])["tier_3"].value_counts()

In [ ]:
# X["tier_3"].value_counts()

In [ ]:
from scipy import stats

inlier_idx = (np.abs(stats.zscore(X)) < 3).all(axis=1)
X = X[inlier_idx]
y = y[inlier_idx]

# Stacking

In [ ]:
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.layers import Dense
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import Adam

from sklearn.metrics import f1_score

In [ ]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from xgboost import XGBClassifier
import lightgbm as lgb
from vecstack import stacking
from sklearn.metrics import f1_score

def f1_score_wrap(y_true, y_pred):
    return f1_score(y_true, y_pred, average='weighted')

import tensorflow_addons as tfa

def NNClf():
    clf = Sequential()
    clf.add(Dense(100, activation='sigmoid', input_dim=23))
    clf.add(Dense(100, activation='sigmoid'))
    clf.add(Dense(100, activation='sigmoid'))
    clf.add(Dense(4, activation='softmax'))
    clf.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=[tfa.metrics.F1Score(average='weighted',num_classes=4)])
    return clf
    
models = [
    KerasClassifier(NNClf,epochs=10, batch_size=500, verbose=0),
    HistGradientBoostingClassifier(),
    lgb.LGBMClassifier(),
    XGBClassifier()
]

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
# X_train, X_test, y_train, y_test = X, X_final, y, []

cat_col = ['tier_2', 'tier_3', 'gender_2']
cont_col = [c for c in selected_columns if c not in cat_col ]

X_train_cont = X_train[cont_col]
X_test_cont = X_test[cont_col]
scaler = StandardScaler()
# scaler = QuantileTransformer(output_distribution='normal')

X_train[cont_col] = scaler.fit_transform(X_train_cont.values)
X_test[cont_col] = scaler.transform(X_test_cont.values)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1734: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

In [ ]:
X_train.shape, y_train.shape, X_test.shape, len(y_test)

((354286, 23), (354286,), (39366, 23), 39366)

In [ ]:
SUBSET_SIZE = -1

S_train, S_test = stacking(models,                     # list of models
                           X_train[:SUBSET_SIZE].values, y_train[:SUBSET_SIZE].values, X_test.values,   # data
                        #    X_train_std, y_train, X_test_std,   # data
                           regression=False,           # classification task (if you need 
                                                       #     regression - set to True)
                           mode='oof_pred_bag',        # mode: oof for train set, predict test 
                                                       #     set in each fold and vote
                           needs_proba=False,          # predict class labels (if you need 
                                                       #     probabilities - set to True) 
                           save_dir=".",              # do not save result and log (to save 
                                                       #     in current dir - set to '.')
                           metric=f1_score_wrap,      # metric: callable
                           n_folds=4,                  # number of folds
                           stratified=True,            # stratified split for folds
                           shuffle=True,               # shuffle the data
                           random_state=0,             # ensure reproducibility
                           verbose=2) 

task:         [classification]
n_classes:    [4]
metric:       [f1_score_wrap]
mode:         [oof_pred_bag]
n_models:     [4]

model  0:     [KerasClassifier]
    fold  0:  [0.70714336]
    fold  1:  [0.68155319]
    fold  2:  [0.72353143]
    fold  3:  [0.68561398]
    ----
    MEAN:     [0.69946049] + [0.01696197]
    FULL:     [0.70318969]

model  1:     [HistGradientBoostingClassifier]
    fold  0:  [0.76147324]
    fold  1:  [0.76343683]
    fold  2:  [0.76116607]
    fold  3:  [0.76088465]
    ----
    MEAN:     [0.76174020] + [0.00100142]
    FULL:     [0.76174185]

model  2:     [LGBMClassifier]
    fold  0:  [0.76661094]
    fold  1:  [0.76777284]
    fold  2:  [0.76788292]
    fold  3:  [0.76729118]
    ----
    MEAN:     [0.76738947] + [0.00050156]
    FULL:     [0.76738658]

model  3:     [XGBClassifier]
    fold  0:  [0.74947944]
    fold  1:  [0.75026480]
    fold  2:  [0.75036135]
    fold  3:  [0.74885028]
    ----
    MEAN:     [0.74973897] + [0.00061664]
    FULL:    

In [ ]:
model = HistGradientBoostingClassifier()

model = model.fit(S_train, y_train[:SUBSET_SIZE])

y_pred = model.predict(S_test)

# print('Final prediction score: [%.8f]' % f1_score(y_test, y_pred, average="weighted"))

In [ ]:
pd.DataFrame(y_pred).to_csv("output.csv",header=False,index=False)

In [ ]:
!pip install tensorflow-addons

#TabNet

In [ ]:
!pip install pytorch-tabnet

In [ ]:
from pytorch_tabnet.tab_model import TabNetClassifier

clf = TabNetClassifier(
    n_d = 32,
    n_a = 32,
    n_steps=5,
) 
clf.fit(
  X_train.values, y_train.values,
  eval_set=[(X_test.values, y_test.values)]
)
preds = clf.predict(X_final.values)

Device used : cuda
epoch 0  | loss: 0.67261 | val_0_accuracy: 0.6942  |  0:00:38s
epoch 1  | loss: 0.61209 | val_0_accuracy: 0.5349  |  0:01:17s
epoch 2  | loss: 0.60428 | val_0_accuracy: 0.70233 |  0:01:56s
epoch 3  | loss: 0.60214 | val_0_accuracy: 0.64604 |  0:02:36s
epoch 4  | loss: 0.60078 | val_0_accuracy: 0.56617 |  0:03:17s
epoch 5  | loss: 0.60003 | val_0_accuracy: 0.28144 |  0:03:58s
epoch 6  | loss: 0.59974 | val_0_accuracy: 0.38782 |  0:04:40s
epoch 7  | loss: 0.59952 | val_0_accuracy: 0.65831 |  0:05:19s
epoch 8  | loss: 0.59837 | val_0_accuracy: 0.70515 |  0:05:57s
epoch 9  | loss: 0.5987  | val_0_accuracy: 0.70335 |  0:06:35s
epoch 10 | loss: 0.59732 | val_0_accuracy: 0.34725 |  0:07:14s
epoch 11 | loss: 0.5968  | val_0_accuracy: 0.7069  |  0:07:52s
epoch 12 | loss: 0.59768 | val_0_accuracy: 0.34733 |  0:08:30s
epoch 13 | loss: 0.59651 | val_0_accuracy: 0.70927 |  0:09:09s
epoch 14 | loss: 0.59457 | val_0_accuracy: 0.51329 |  0:09:48s
epoch 15 | loss: 0.59382 | val_0_acc

In [ ]:
preds

array([4, 4, 4, ..., 1, 4, 4])